## --- Day 5: Print Queue ---
Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new **sleigh launch safety manual** updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a **very specific order**. The notation `X|Y` means that if both page number `X` and page number `Y` are to be produced as part of an update, page number `X` **must** be printed at some point before page number `Y`.

The Elf has for you both the **page ordering rules** and the **pages to produce in each update** (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

```shell
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47

```

The first section specifies the **page ordering rules**, one per line. The first rule, `47|53`, means that if an update includes both page number 47 and page number 53, then page number 47 **must** be printed at some point before page number 53. (47 doesn't necessarily need to be **immediately** before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each **update**. Because most safety manuals are different, the pages needed in the updates are different too. The first update, `75,47,61,53,29`, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying **which updates are already in the right order**.

In the above example, the first update (`75,47,61,53,29`) is in the right order:


- `75` is correctly first because there are rules that put each other page after it: `75|47`, `75|61`, `75|53`, and `75|29`.

- `47` is correctly second because 75 must be before it (`75|47`) and every other page must be after it according to `47|61`, `47|53`, and `47|29`.

- `61` is correctly in the middle because 75 and 47 are before it (`75|61` and `47|61`) and 53 and 29 are after it (`61|53` and `61|29`).

- `53` is correctly fourth because it is before page number 29 (`53|29`).

- `29` is the only page left and so is correctly last.



Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, `75,97,47,61,53`, is **not** in the correct order: it would print 75 before 97, which violates the rule `97|75`.

The fifth update, `61,13,29`, is also **not** in the correct order, since it breaks the rule `29|13`.

The last update, `97,13,75,29,47`, is **not** in the correct order due to breaking several rules.

For some reason, the Elves also need to know the **middle page number** of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

```shell
75,47,61,53,29
97,61,53,29,13
75,29,13

```

These have middle page numbers of `61`, `53`, and `29` respectively. Adding these page numbers together gives `143`.

Of course, you'll need to be careful: the actual list of **page ordering rules** is bigger and more complicated than the above example.

Determine which updates are already in the correct order. **What do you get if you add up the middle page number from those correctly-ordered updates?**



In [22]:
test = '''
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47

'''

def parse(s: str) -> list[tuple[tuple[int]], list[list[int]]]:
    rules, updates = s.strip().split('\n\n')
    return (
        tuple(tuple(int(p) for p in v.strip().split('|')) for v in rules.split('\n')),
        [[int(p) for p in v.strip().split(',')] for v in updates.split('\n')]
    )
    
def check_update(update: list[int], rules: tuple[tuple[int]]) -> tuple[int, int] | None:
    for i, p in enumerate(update):
        for b, a in rules:
            if p == a and b in update:
                bi = update.index(b)
                if i < bi: return (bi, i)
                
            if p == b and a in update:
                ai = update.index(a)
                if i > ai: return (i, ai)
                
    return None
        
def validate_updates(updates: list[list[int]], rules: tuple[tuple[int]], autofix: bool = False) -> int:
    checksum = 0
    for update in updates:
        while autofix:
            if fix := check_update(update, rules):
                update.insert(fix[1], update.pop(fix[0]))
            else:
                break
    
        if check_update(update, rules) == None:
            checksum += update[len(update) // 2]
            
    return checksum
    
rules, updates = parse(test)
checksum_test = validate_updates(updates, rules) 
checksum_test

143

In [19]:
with open('day5.input') as f:
    rules, updates = parse(f.read())
    
checksum = validate_updates(updates, rules)
checksum

4689

## --- Part Two ---
While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the **incorrectly-ordered updates**, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:


- `75,97,47,61,53` becomes `97,75,47,61,53`.

- `61,13,29` becomes `61,29,13`.

- `97,13,75,29,47` becomes `97,75,47,29,13`.



After taking **only the incorrectly-ordered updates** and ordering them correctly, their middle page numbers are `47`, `29`, and `47`. Adding these together produces `123`.

Find the updates which are not in the correct order. **What do you get if you add up the middle page numbers after correctly ordering just those updates?**



In [23]:
rules, updates = parse(test)
checksum_corrected_test = validate_updates(updates, rules, True) - checksum_test
checksum_corrected_test

[97, 75, 47, 61, 53] 1
[61, 29, 13] 1
[97, 29, 13, 75, 47] 1
[97, 75, 29, 13, 47] 2
[97, 75, 47, 29, 13] 3


123

In [24]:
with open('day5.input') as f:
    rules, updates = parse(f.read())
    
checksum_corrected = validate_updates(updates, rules, True) - checksum
checksum_corrected

[71, 74, 56, 86, 81, 84, 44, 53, 92, 12, 36, 15, 66, 95, 26] 1
[36, 71, 74, 56, 86, 81, 84, 44, 53, 92, 12, 15, 66, 95, 26] 2
[84, 36, 71, 74, 56, 86, 81, 44, 53, 92, 12, 15, 66, 95, 26] 3
[84, 12, 36, 71, 74, 56, 86, 81, 44, 53, 92, 15, 66, 95, 26] 4
[84, 12, 86, 36, 71, 74, 56, 81, 44, 53, 92, 15, 66, 95, 26] 5
[84, 12, 86, 36, 71, 95, 74, 56, 81, 44, 53, 92, 15, 66, 26] 6
[84, 12, 86, 36, 71, 81, 95, 74, 56, 44, 53, 92, 15, 66, 26] 7
[84, 12, 86, 36, 71, 81, 95, 74, 53, 56, 44, 92, 15, 66, 26] 8
[84, 12, 86, 36, 71, 81, 95, 74, 53, 56, 26, 44, 92, 15, 66] 9
[84, 12, 86, 36, 71, 81, 95, 74, 53, 56, 66, 26, 44, 92, 15] 10
[84, 12, 86, 36, 71, 81, 95, 74, 53, 56, 66, 26, 15, 44, 92] 11
[26, 24, 68, 47, 42, 73, 41, 52, 44, 78, 64, 76, 29, 82, 38] 1
[26, 44, 24, 68, 47, 42, 73, 41, 52, 78, 64, 76, 29, 82, 38] 2
[26, 44, 24, 52, 68, 47, 42, 73, 41, 78, 64, 76, 29, 82, 38] 3
[26, 44, 24, 52, 68, 47, 42, 29, 73, 41, 78, 64, 76, 82, 38] 4
[26, 44, 24, 52, 68, 47, 42, 41, 29, 73, 78, 64, 76, 

[73, 81, 96, 36, 71, 12, 74, 32, 43, 63, 86, 69, 84, 59, 93, 11, 95, 37, 18, 78, 34] 1
[78, 73, 81, 96, 36, 71, 12, 74, 32, 43, 63, 86, 69, 84, 59, 93, 11, 95, 37, 18, 34] 2
[78, 73, 32, 81, 96, 36, 71, 12, 74, 43, 63, 86, 69, 84, 59, 93, 11, 95, 37, 18, 34] 3
[78, 73, 32, 37, 81, 96, 36, 71, 12, 74, 43, 63, 86, 69, 84, 59, 93, 11, 95, 18, 34] 4
[78, 73, 32, 59, 37, 81, 96, 36, 71, 12, 74, 43, 63, 86, 69, 84, 93, 11, 95, 18, 34] 5
[78, 73, 32, 84, 59, 37, 81, 96, 36, 71, 12, 74, 43, 63, 86, 69, 93, 11, 95, 18, 34] 6
[78, 73, 32, 43, 84, 59, 37, 81, 96, 36, 71, 12, 74, 63, 86, 69, 93, 11, 95, 18, 34] 7
[78, 73, 32, 93, 43, 84, 59, 37, 81, 96, 36, 71, 12, 74, 63, 86, 69, 11, 95, 18, 34] 8
[78, 73, 32, 18, 93, 43, 84, 59, 37, 81, 96, 36, 71, 12, 74, 63, 86, 69, 11, 95, 34] 9
[78, 73, 32, 18, 93, 69, 43, 84, 59, 37, 81, 96, 36, 71, 12, 74, 63, 86, 11, 95, 34] 10
[78, 73, 32, 18, 93, 69, 43, 84, 59, 11, 37, 81, 96, 36, 71, 12, 74, 63, 86, 95, 34] 11
[78, 73, 32, 18, 93, 69, 43, 84, 59, 12, 

6336